In [1]:
import numpy as np
import tensorflow as tf
import time
import os
import math
import pandas as pd
from tqdm.notebook import tqdm
import scipy.sparse as sp
from tffm import TFFMClassifier
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics import roc_auc_score, accuracy_score

dir_ = '../../HPCF/all/test_0.8/data/'
file_name = 'normalized_log_filter_track_5_user_100.csv'

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site

In [2]:
# Note that x is a 2D numpy array, 
# x[i, :] contains the user-item pair, and y[i] is the corresponding rating.

df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
df_train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name[:-3] + 'pkl'))
train_y = np.loadtxt(os.path.join(dir_, 'train_y_' + file_name[:-3] + 'csv'), delimiter=',')


num_users = len(df['uid'].unique())
num_tracks = len(df['tid'].unique())
print(num_users, num_tracks)

953 157567


In [3]:
unique_users = len(df['uid'].unique())
unique_tracks = len(df['tid'].unique())
l = unique_users + unique_tracks + 1
tid_start = unique_users
tag_start = unique_users + unique_tracks
print ('Tid start position : ' + str(tid_start))
print ('Tag start position : ' + str(tag_start))
print ('Length of vector : ' + str(l))
print('----------------------------------------')
print ('Number of unique users : ' + str(unique_users))
print ('Number of unique tracks : ' + str(unique_tracks))

Tid start position : 953
Tag start position : 158520
Length of vector : 158521
----------------------------------------
Number of unique users : 953
Number of unique tracks : 157567


In [4]:
# pop = {}
# for i in df['tid'].unique():
#     pop[i] = len(df[df['tid']==i])/unique_users

In [5]:
import pickle
with open(os.path.join(dir_,'pop_dict.pickle'), 'rb') as handle:
    pop = pickle.load(handle)
pop[0], len(df[df['tid']==0])/unique_users

(0.005246589716684155, 0.005246589716684155)

In [6]:
# pop
i = 0
v = []
sp_rows = []
sp_cols = []
for index, row in tqdm(df_train.iterrows(), total=len(df_train)):
    sp_rows.append(i)
    sp_cols.append(row['uid'])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row['tid'])
    v.append(1)

    sp_rows.append(i)
    sp_cols.append(tag_start)
    v.append(pop[row['tid']])
        
    i += 1

train_x = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(df_train), l), dtype=float)

In [7]:
order = 2
model = TFFMRegressor(
    order=order, 
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
    n_epochs=10, 
    batch_size=16384,
    init_std=0.00001,
    reg=0.0001,
    input_type='sparse'
)
model.fit(train_x, train_y, show_progress=True)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





100%|██████████| 10/10 [00:25<00:00,  2.56s/epoch]


In [8]:
train_x = 0

In [9]:
track_ids = []
for i in range(unique_tracks):
    track_ids.append(i)

all_tracks = pd.DataFrame()
all_tracks['tid'] = track_ids
all_tracks['count'] = 0

In [10]:
x_test = []
for i in tqdm(range(unique_users)):
    user = df_train[df_train['uid']==i]
    top_n = all_tracks.set_index('tid').add(user.set_index('tid'), fill_value=0).reset_index()
    top_n = top_n[top_n['count']==0]
    top_n['uid'] = i
    top_n = top_n[['uid', 'tid']]
    top_n = top_n.values.tolist()
    x_test.extend(top_n)

In [34]:
i = 0
v = []
sp_rows = []
sp_cols = []
for row in tqdm(x_test[:int(len(x_test)/4*1)]):
    sp_rows.append(i)
    sp_cols.append(row[0])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row[1])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tag_start)
    v.append(pop[row[1]])
    
    i += 1

In [35]:
test_x = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(int(len(v)/3), l), dtype=float)

In [36]:
v = []
sp_rows = []
sp_cols = []

In [37]:
pred1 = model.predict(test_x)

In [31]:
len(pred1), pred1

(36924410,
 array([0.31861806, 0.42533082, 0.32368976, ..., 0.32251853, 0.36477205,
        0.29812625], dtype=float32))

In [38]:
import pickle
open_file = open(os.path.join('../data/all (log)(user)', 'pred1.pkl'), "wb")
pickle.dump(pred4, open_file)
open_file.close()

In [39]:
test_x = 0

In [40]:
pred1 = list(pred1)
pred2 = list(pred2)
pred3 = list(pred3)
pred4 = list(pred4)

In [41]:
pred1.extend(pred2)
print(len(pred1))
pred1.extend(pred3)
print(len(pred1))
pred1.extend(pred4)
print(len(pred1))

73848820
110773230
147697641


In [42]:
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', pred1, False) 
df2[:10]

,uid,tid,rating
0,0,4,0.337096
1,0,14,0.472648
2,0,19,0.398296
3,0,21,0.364385
4,0,27,0.331364
5,0,35,0.394582
6,0,36,0.413474
7,0,41,0.290710
8,0,44,0.402797
9,0,46,0.608555


In [43]:
df2.to_pickle(os.path.join('../data/all (log)(user)', 'topN_pred_' + file_name[:-3] + 'pkl'))

In [44]:
print(len(x_test))

147697641


In [33]:
dir_ = '../data/'
folder = 'sp_matrix_pop(old)'
file_name = 'normalized_to_rating_filter_track_5_user_100.pkl'

test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
7,0,3,0.25,1
11,0,7,2.50,3
32,0,18,1.50,2
34,0,20,2.50,3
45,0,31,5.00,17


In [34]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(unique_users)):
    prediction = df2[df2['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

evaluation = []
satisfication = 0
for i in tqdm(range(unique_users)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/unique_users)


0.24477272727272723
